In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar3/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 50
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.3.6"
merge_list = ""


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.3.6.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

28395

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

28395

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 28395
topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

0.0651237964630127

In [18]:
word_doc_matrix

<28395x1938 sparse matrix of type '<class 'numpy.float64'>'
	with 23770 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:132: UserWarning: A large number of your vertices were disconnected from the manifold.
Disconnection_distance = 1 has removed 458496 edges.
It has fully disconnected 11194 vertices.
You might consider using find_disconnected_points() to find and remove these points from your data.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 11min 53s, sys: 4min 15s, total: 16min 9s
Wall time: 6min 4s


In [21]:
print(runtime_umap)

364.0739767551422


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 4min 5s, sys: 331 ms, total: 4min 5s
Wall time: 3min 26s


In [23]:
print(runtime_umap_transform)

206.32475233078003


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_embedding_t2.3.6.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_clusterable_embedding_t2.3.6.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
4,hdb,t2.3.6,1,ss250cs50,50,250,0.445156,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
4,hdb,t2.3.6,1,ss250cs50,50,250,0.445156,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.3.6,1,ss50cs450,450,50,0.302185,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
73,hdb,t2.3.6,1,ss100cs450,450,100,0.341414,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
74,hdb,t2.3.6,1,ss150cs450,450,150,0.404088,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
75,hdb,t2.3.6,1,ss200cs450,450,200,0.438434,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
4,hdb,t2.3.6,1,ss250cs50,50,250,0.445156,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.3.6,1,ss100cs450,450,100,0.341414,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
74,hdb,t2.3.6,1,ss150cs450,450,150,0.404088,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
75,hdb,t2.3.6,1,ss200cs450,450,200,0.438434,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
76,hdb,t2.3.6,1,ss250cs450,450,250,0.463550,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
4,hdb,t2.3.6,1,ss250cs50,50,250,0.445156,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.3.6,1,ss150cs450,450,150,0.404088,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
75,hdb,t2.3.6,1,ss200cs450,450,200,0.438434,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
76,hdb,t2.3.6,1,ss250cs450,450,250,0.463550,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
77,hdb,t2.3.6,1,ss300cs450,450,300,0.513955,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
4,hdb,t2.3.6,1,ss250cs50,50,250,0.445156,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.3.6,1,ss200cs450,450,200,0.438434,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
76,hdb,t2.3.6,1,ss250cs450,450,250,0.463550,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
77,hdb,t2.3.6,1,ss300cs450,450,300,0.513955,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
78,hdb,t2.3.6,1,ss350cs450,450,350,0.591937,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.3.6,1,ss50cs50,50,50,0.847438,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
1,hdb,t2.3.6,1,ss100cs50,50,100,0.360054,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
2,hdb,t2.3.6,1,ss150cs50,50,150,0.385113,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
3,hdb,t2.3.6,1,ss200cs50,50,200,0.429694,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
4,hdb,t2.3.6,1,ss250cs50,50,250,0.445156,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.3.6,1,ss250cs450,450,250,0.463550,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
77,hdb,t2.3.6,1,ss300cs450,450,300,0.513955,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
78,hdb,t2.3.6,1,ss350cs450,450,350,0.591937,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."
79,hdb,t2.3.6,1,ss400cs450,450,400,0.616693,"rt_vect: 0.0651237964630127, min_df: 4, max_df...","hellinger, rt_umap_tf: 206.32475233078003, rt_..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,google/chaps-linux,-1,-1,-1,-1,-1,-1,-1,4,3,...,2,-1,-1,-1,-1,-1,-1,-1,2,2
1,droidefense/engine,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,wdgdc/Jarvis,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,PaloAltoNetworks/google-cloud-hub-spoke-tutorial,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,liwenhe1993/charts,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,sakhnyuk/javascript-interview-questions,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
28391,mohammaduzair9/Face-Recognition,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
28392,deepinx/age-gender-estimation,8,11,9,7,7,7,6,4,3,...,2,3,3,3,3,3,3,2,2,2
28393,aws-cloudformation/aws-cloudformation-resource...,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_labels_results_t2.3.6.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
4,kmeans-umapt,t2.3.6,1,22,11194,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.242167,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


...

,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
4,kmeans-umapt,t2.3.6,1,22,11194,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.242167,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
5,kmeans-umapt,t2.3.6,1,25,11194,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.269583,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
6,kmeans-umapt,t2.3.6,1,28,11194,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.354918,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
7,kmeans-umapt,t2.3.6,1,31,11194,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.330519,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
8,kmeans-umapt,t2.3.6,1,34,11194,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.351372,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
9,kmeans-umapt,t2.3.6,1,37,11194,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.375835,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
4,kmeans-umapt,t2.3.6,1,22,11194,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.242167,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
5,kmeans-umapt,t2.3.6,1,25,11194,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.269583,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
6,kmeans-umapt,t2.3.6,1,28,11194,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.354918,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
7,kmeans-umapt,t2.3.6,1,31,11194,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.330519,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
8,kmeans-umapt,t2.3.6,1,34,11194,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.351372,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
9,kmeans-umapt,t2.3.6,1,37,11194,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.375835,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
4,kmeans-umapt,t2.3.6,1,22,11194,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.242167,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
5,kmeans-umapt,t2.3.6,1,25,11194,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.269583,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
6,kmeans-umapt,t2.3.6,1,28,11194,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.354918,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
7,kmeans-umapt,t2.3.6,1,31,11194,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.330519,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
8,kmeans-umapt,t2.3.6,1,34,11194,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.351372,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
9,kmeans-umapt,t2.3.6,1,37,11194,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.375835,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
4,kmeans-umapt,t2.3.6,1,22,11194,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.242167,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
5,kmeans-umapt,t2.3.6,1,25,11194,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.269583,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
6,kmeans-umapt,t2.3.6,1,28,11194,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.354918,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
7,kmeans-umapt,t2.3.6,1,31,11194,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.330519,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
8,kmeans-umapt,t2.3.6,1,34,11194,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.351372,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
9,kmeans-umapt,t2.3.6,1,37,11194,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.375835,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.3.6,1,10,11194,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.283175,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
1,kmeans-umapt,t2.3.6,1,13,11194,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.156239,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
2,kmeans-umapt,t2.3.6,1,16,11194,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.171033,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
3,kmeans-umapt,t2.3.6,1,19,11194,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.206382,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
4,kmeans-umapt,t2.3.6,1,22,11194,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.242167,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
5,kmeans-umapt,t2.3.6,1,25,11194,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.269583,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
6,kmeans-umapt,t2.3.6,1,28,11194,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.354918,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
7,kmeans-umapt,t2.3.6,1,31,11194,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.330519,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
8,kmeans-umapt,t2.3.6,1,34,11194,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.351372,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."
9,kmeans-umapt,t2.3.6,1,37,11194,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.375835,"(hellinger, rt_umap_tf: 206.32475233078003, rt..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,google/chaps-linux,4,3,6,4,8,6,17,5,29,11,17,7,33,8
6,JeffreyRuder/Kitchen,0,0,1,0,2,0,0,0,0,0,2,2,0,2
7,Azure/ai-solution-accelerators-list,4,3,12,4,8,20,4,23,18,5,25,15,13,25
11,intel/ipp-crypto,8,1,13,1,4,22,7,6,12,36,12,5,11,43
13,WILL-Hunting/FakeXiecheng.API,2,12,2,12,6,12,12,21,21,30,33,20,40,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28388,samuel-code/face-detection,2,9,14,5,1,23,26,7,14,26,20,33,37,22
28389,ApiO/jquery.triSwitch,6,4,15,2,13,5,23,28,20,19,30,11,16,13
28391,mohammaduzair9/Face-Recognition,6,10,5,14,3,10,14,4,10,33,15,37,17,34
28392,deepinx/age-gender-estimation,0,0,1,0,2,0,0,0,0,0,2,2,0,2


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_labels_results_t2.3.6.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.15706


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526
4,kmeans,t2.3.6,1,22,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.300606


...

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526
4,kmeans,t2.3.6,1,22,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.300606
5,kmeans,t2.3.6,1,25,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.346418
6,kmeans,t2.3.6,1,28,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.379934
7,kmeans,t2.3.6,1,31,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.415289
8,kmeans,t2.3.6,1,34,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.449747
9,kmeans,t2.3.6,1,37,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.480088


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526
4,kmeans,t2.3.6,1,22,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.300606
5,kmeans,t2.3.6,1,25,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.346418
6,kmeans,t2.3.6,1,28,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.379934
7,kmeans,t2.3.6,1,31,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.415289
8,kmeans,t2.3.6,1,34,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.449747
9,kmeans,t2.3.6,1,37,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.480088


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526
4,kmeans,t2.3.6,1,22,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.300606
5,kmeans,t2.3.6,1,25,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.346418
6,kmeans,t2.3.6,1,28,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.379934
7,kmeans,t2.3.6,1,31,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.415289
8,kmeans,t2.3.6,1,34,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.449747
9,kmeans,t2.3.6,1,37,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.480088


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526
4,kmeans,t2.3.6,1,22,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.300606
5,kmeans,t2.3.6,1,25,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.346418
6,kmeans,t2.3.6,1,28,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.379934
7,kmeans,t2.3.6,1,31,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.415289
8,kmeans,t2.3.6,1,34,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.449747
9,kmeans,t2.3.6,1,37,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.480088


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.3.6,1,10,10,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.157060
1,kmeans,t2.3.6,1,13,13,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.192712
2,kmeans,t2.3.6,1,16,16,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.226606
3,kmeans,t2.3.6,1,19,19,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.258526
4,kmeans,t2.3.6,1,22,22,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.300606
5,kmeans,t2.3.6,1,25,25,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.346418
6,kmeans,t2.3.6,1,28,28,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.379934
7,kmeans,t2.3.6,1,31,31,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.415289
8,kmeans,t2.3.6,1,34,34,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.449747
9,kmeans,t2.3.6,1,37,37,"rt_vect: 0.0651237964630127, min_df: 4, max_df...",0.480088


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,google/chaps-linux,1,3,6,13,5,1,13,1,4,5,31,31,40,19
1,droidefense/engine,1,3,1,0,5,1,2,2,2,5,0,2,1,0
2,wdgdc/Jarvis,1,3,1,0,5,1,2,2,2,5,0,2,1,0
3,PaloAltoNetworks/google-cloud-hub-spoke-tutorial,1,3,1,0,5,1,2,2,2,5,0,2,1,0
4,liwenhe1993/charts,1,3,1,0,5,1,2,2,2,5,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,sakhnyuk/javascript-interview-questions,1,3,1,0,5,1,2,2,2,5,0,2,1,0
28391,mohammaduzair9/Face-Recognition,1,3,1,12,5,1,2,2,2,12,0,2,16,0
28392,deepinx/age-gender-estimation,1,3,1,0,5,1,2,2,2,5,0,2,1,0
28393,aws-cloudformation/aws-cloudformation-resource...,1,3,1,0,5,1,2,2,2,5,0,2,1,0


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_labels_results_t2.3.6.pkl
